In [1]:
import pandas as pd
#import cv2
import random

In [70]:
label_path = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/label/"
video_files = pd.read_csv(label_path + "combined_labels_v3_cleaned.csv")

video_list = video_files["video_name"].unique().tolist()
video_list[0:3]

['OC01_R_10s_lowfps.mp4', 'OC06_R_10s_lowfps.mp4', 'OC101_R_10s_lowfps.mp4']

In [71]:
#shuffling the list with seed
random.seed(4)
random.shuffle(video_list)
video_list[0:3]

['P53_R_10s_lowfps.mp4', 'P36_R_10s_lowfps.mp4', 'OC08_R_10s_lowfps.mp4']

In [72]:
## Splitting the videos between train-valid-test
total_files = len(video_list)
train_len = int(total_files* 0.8)
test_len = int(total_files* 0.1)
valid_len = total_files - (train_len + test_len)
print(train_len, valid_len, test_len)

train_videos = video_list[0:train_len]
valid_videos = video_list[train_len:train_len+valid_len]
test_videos = video_list[train_len+valid_len: ]  #[train_len:train_len+valid_len]

50 7 6


In [117]:
import cv2
import os
#video_name = 'OC07_R_10s_lowfps.mp4'
input_video_path = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/cropped_videos/'
output_path = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/"

# using label path as some of the videos have last frame missing in labels (unsure why they left 1 frame in some of the videos)
label_path = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/label/"
video_files = pd.read_csv(label_path + "combined_labels_v3_cleaned.csv")


# Extracting images
for folder in ["Train", "Test", "Valid"]:
    if folder == "Train":
        folder_videos = train_videos
    elif folder == "Test":
        folder_videos = test_videos
    else: 
        folder_videos = valid_videos
    for video_name in folder_videos:
        cap = cv2.VideoCapture( input_video_path + video_name)
        success,image = vidcap.read()
        count = 0
        labeled_count = len(video_files[video_files["video_name"]== video_name])
        while (vidcap.isOpened() and count<=labeled_count-1):
            ret,image = cap.read()
            if ret==True:
                cv2.imwrite(os.path.join(output_path+ folder,f"{video_name}@{count}.jpg"), image)     # save frame as JPEG file
                count += 1
                if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                    break
            else:
                break

    cap.release()
    cv2.destroyAllWindows()

In [60]:
## Update : 16-07 : we need to clean up few image files for which labels arent available & also those where coordinates are negative
#### Cleaning up images for which there is one or more missing labels in the label file

root_image_path = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/"

label_path = root_image_path + "label/"
video_files = pd.read_csv(label_path + "combined_labels_v3_cleaned.csv")
missing_data = video_files[video_files[["new_finger_x","new_finger_y","new_thumb_x","new_thumb_y"]].isnull().any(axis=1)]

missing_data["filename"] = missing_data["video_name"] + "@" + missing_data.seq.str[3:6].astype(int).astype(str)

missing_data

<ipython-input-60-41989dfaf0d9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_data["filename"] = missing_data["video_name"] + "@" + missing_data.seq.str[3:6].astype(int).astype(str)


video_name         seq  new_finger_x  new_finger_y  \
4798   OC50_R_10s_lowfps.mp4  img025.png           NaN           NaN   
5001    P06_R_10s_lowfps.mp4  img062.png           NaN           NaN   
5313    P23_R_10s_lowfps.mp4  img044.png           NaN           NaN   
6257   YC02_R_10s_lowfps.mp4  img165.png           NaN           NaN   
6442  OC100_R_10s_lowfps.mp4  img020.png           NaN           NaN   
...                      ...         ...           ...           ...   
7948  YC100_R_10s_lowfps.mp4  img046.png           NaN           NaN   
7961  YC100_R_10s_lowfps.mp4  img059.png           NaN           NaN   
8030  YC100_R_10s_lowfps.mp4  img128.png           NaN           NaN   
8031  YC100_R_10s_lowfps.mp4  img129.png           NaN           NaN   
8459   OC10_R_10s_lowfps.mp4  img063.png           NaN           NaN   

      new_thumb_x  new_thumb_y                    filename  
4798          NaN          NaN    OC50_R_10s_lowfps.mp4@25  
5001          NaN          NaN     P06_R_10s_lowfps.mp4@62  
5313          NaN          NaN     P23_R_10s_lowfps.mp4@44  
6257          NaN          NaN   YC02_R_10s_lowfps.mp4@165  
6442   152.376836   442.820897   OC100_R_10s_lowfps.mp4@20  
...           ...          ...                         ...  
7948   276.468460   357.618120   YC100_R_10s_lowfps.mp4@46  
7961   261.626160   369.986704   YC100_R_10s_lowfps.mp4@59  
8030   236.888992   397.197588  YC100_R_10s_lowfps.mp4@128  
8031   229.467842   409.566172  YC100_R_10s_lowfps.mp4@129  
8459   238.322830   464.983934    OC10_R_10s_lowfps.mp4@63  

[114 rows x 7 columns]

In [62]:
## Deleting the above files from image folders

for folder in ["Train","Valid", "Test"]:
    for bad_file in missing_data["filename"].unique():
        try:
            os.remove(root_image_path+folder + "/" + bad_file +".jpg")
        except :
            continue

In [69]:
# Verifying that the files have been removed
for folder in ["Train","Valid", "Test"]:
    for bad_file in missing_data["filename"].unique():
        for cur_files in os.listdir(root_image_path+folder): 
            if cur_files == bad_file + ".jpg":
                print(cur_files)


In [70]:
# Just downloading the names of bad files - not needed though. but will be handy 
#missing_data.to_csv("bad_files.csv")

In [106]:
## Now Checking why P06 has 4 negative labels. 
# import cv2
image = cv2.imread("C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/labeled_data/student3_labelled/OC05_R_10s_lowfps/img000.png")
image = cv2.circle(image, (751,500), radius=10, color=(0, 0, 255), thickness=-1)
cv2.imwrite("result_check.png",image)

In [102]:
## Seems the averaging method of bounding box could not capture these frames properly. 
# So will have to remove these negative frames as well
negative_files = video_files[video_files["new_finger_x"]<0]
negative_files = negative_files.append(video_files[video_files["new_finger_y"]<0])
negative_files = negative_files.append(video_files[video_files["new_thumb_x"]<0])
negative_files = negative_files.append(video_files[video_files["new_thumb_y"]<0])

negative_files["filename"] = negative_files["video_name"] + "@" + negative_files.seq.str[3:6].astype(int).astype(str)

print(negative_files)
print(len(negative_files))

                video_name         seq  new_finger_x  new_finger_y  \
4939  P06_R_10s_lowfps.mp4  img000.png     -2.657505    329.868469   
4942  P06_R_10s_lowfps.mp4  img003.png     -8.093929    332.586681   
4943  P06_R_10s_lowfps.mp4  img004.png     -8.093929    338.023105   
4944  P06_R_10s_lowfps.mp4  img005.png     -2.657505    329.868469   
4947  P06_R_10s_lowfps.mp4  img008.png     -5.375717    367.923437   
5449  P50_R_10s_lowfps.mp4  img015.png    273.380248    -22.117638   
5461  P50_R_10s_lowfps.mp4  img027.png    243.479915     -8.526578   
5477  P50_R_10s_lowfps.mp4  img043.png    248.916339    -11.244790   
5481  P50_R_10s_lowfps.mp4  img047.png    259.789188    -22.117638   
5485  P50_R_10s_lowfps.mp4  img051.png    251.634551    -16.681214   
5527  P50_R_10s_lowfps.mp4  img093.png    219.016007     -3.090154   
5562  P50_R_10s_lowfps.mp4  img128.png    243.479915     -3.090154   
5567  P50_R_10s_lowfps.mp4  img133.png    238.043491     -0.371942   

      new_thumb_x  

In [103]:
## Deleting the above 0ve coordinate files from image folders
for folder in ["Train","Valid", "Test"]:
    for bad_file in negative_files["filename"].unique():
        try:
            os.remove(root_image_path+folder + "/" + bad_file +".jpg")
        except :
            continue


print("all done")

all done


In [127]:
# Verifying that the files have been removed
for folder in ["Train","Valid", "Test"]:
    for bad_file in negative_files["filename"].unique():
        for cur_files in os.listdir(root_image_path+folder): 
            if cur_files == bad_file + ".jpg":
                print(cur_files)



In [142]:
## Update 16-7 21:15
# not sure why colab is still throwing error as it finds some image which does not have a label. So checking which files are still present - even after cleanup

train_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/Train/' 
valid_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/Valid/'
test_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/test/'



train_list = os.listdir(train_directory)
test_list = os.listdir(test_directory)
valid_list = os.listdir(valid_directory)

import csv

with open("train_files.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(train_list)
    
with open("test_files.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(test_list)
    
with open("valid_files.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(valid_list)

In [156]:
## Need to delete these image files checked in excel workbook : Issue with labels_14_july.csv
manual_delete_list = ["OC05_R_10s_lowfps.mp4@8","OC16_R_10s_lowfps.mp4@3","OC50_R_10s_lowfps.mp4@5","P11_R_10s_lowfps.mp4@11","YC02_R_10s_lowfps.mp4@0"]


for folder in ["Train","Valid", "Test"]:
    for bad_file in manual_delete_list:
        try:
            os.remove(root_image_path+folder + "/" + bad_file +".jpg")
        except :
            continue


In [157]:
#video_files.seq

video_files["filename"] = video_files["video_name"] + "@" + video_files.seq.str[3:6].astype(int).astype(str)

video_files[video_files["filename"] == "OC01_R_10s_lowfps.mp4@2"]["new_thumb_x"]

2    229.9468
Name: new_thumb_x, dtype: float64

In [126]:
### Delete this after testing 
# just checking if labels of adjusted files look fine 


# change the directories here when running on server/cloud - Kanav
train_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/Train/' 
valid_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/Test/'
label_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/label/'

labels_data = pd.read_csv(label_directory + "combined_labels_v3_cleaned.csv")

image_name = "YC103_R_10s_lowfps.mp4@142.jpg"

# image: ndarray keypoints: ndarray but not normalized
image = cv2.imread(train_directory + image_name)

## Resizing the image to a square dimension - Kanav
old_size = image.shape[:2]
desired_size = max(old_size)
ratio = float(desired_size)/max(old_size)
new_size = tuple([int(x*ratio) for x in old_size])
new_size
delta_w = desired_size - new_size[1]
delta_h = desired_size - new_size[0]
top, bottom = delta_h//2, delta_h-(delta_h//2)
left, right = delta_w//2, delta_w-(delta_w//2)

color = [0,0,0]
image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,
    value=color)

#resizing image as per model requirements - change this when you try some other size - Kanav        
image = cv2.resize(image, (128, 128), interpolation=cv2.INTER_LINEAR)

# Initialize frame counter - kanav - may not be needed now
#cnt = 0 
filename = image_name.split("@")[0]
file_index = int(image_name.split("@")[1].split(".")[0])

temp_labels = labels_data[labels_data["video_name"] == filename]
keypoints = []

#adjusting the width labels as per new image dimensions of a square
x1 = int(temp_labels.iloc[file_index,2]) + left
y1 = int(temp_labels.iloc[file_index,3]) + top
x2 = int(temp_labels.iloc[file_index,4]) + left
y2 = int(temp_labels.iloc[file_index,5]) + top

#adjusting labels as per dimension of reduced size for model
x1 = int(x1*128/int(max(old_size)) )
y1 = int(y1*128/int(max(old_size)) )
x2 = int(x2*128/int(max(old_size)) )
y2 = int(y2*128/int(max(old_size)) )

img = cv2.circle(image, (x1,y1), radius=1, color=(0, 0, 255), thickness=-1)
cv2.imwrite("result_check_finger.png",img)

True

In [123]:
# orig = cv2.imread(train_directory + image_name)
# img = cv2.circle(orig, (433,117), radius=2, color=(0, 0, 255), thickness=-1)
# cv2.imwrite("result_orig_check_finger.png",orig)

True

In [124]:
print("all good")

all good
